<a href="https://colab.research.google.com/github/Fidelisaboke/robust-nids/blob/feat%2Fbaseline-model/notebooks/03_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing the TII-SSRC-23 Dataset

## Mount Google Drive

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import required libraries

In [40]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

## Constants and configuration

In [26]:
# Paths
BASE_DIR = "/content/drive/MyDrive/Datasets/TII-SSRC-23/"
SAMPLE_CSV_PATH = os.path.join(BASE_DIR, "sampled_200k_simple.csv")
LOCAL_DATA_DIR = os.path.join(BASE_DIR, "Additional/")
CLEAN_OUTPUT_CSV = os.path.join(BASE_DIR, "cleaned_data.csv")
CLEAN_OUTPUT_PARQUET = os.path.join(BASE_DIR, "cleaned_data.parquet")

# Local file definitions
LOCAL_CSV_LABELS = {
    "Benign_Video.csv": ("Benign", "Video", "Video"),
    "Benign_Audio.csv": ("Benign", "Audio", "Audio"),
    "Benign_Text.csv": ("Benign", "Text", "Text"),
    "Benign_Web.csv": ("Benign", "Browsing", "HTTPS"),
    "InfoGathering.csv": ("Malicious", "Information Gathering", "Port Scan"),
    "Bruteforce.csv": ("Malicious", "Bruteforce", "Bruteforce SSH")
}

# Column Mapping (CICFlowMeter -> TII naming convention)
COL_MAPPING = {
    "flow_duration": "Flow Duration", "flow_byts_s": "Flow Bytes/s", "flow_pkts_s": "Flow Packets/s",
    "flow_iat_mean": "Flow IAT Mean", "flow_iat_max": "Flow IAT Max", "flow_iat_min": "Flow IAT Min",
    "flow_iat_std": "Flow IAT Std", "fwd_pkts_s": "Fwd Packets/s", "tot_fwd_pkts": "Total Fwd Packet",
    "totlen_fwd_pkts": "Total Length of Fwd Packet", "fwd_pkt_len_max": "Fwd Packet Length Max",
    "fwd_pkt_len_min": "Fwd Packet Length Min", "fwd_pkt_len_mean": "Fwd Packet Length Mean",
    "fwd_pkt_len_std": "Fwd Packet Length Std", "fwd_iat_tot": "Fwd IAT Total", "fwd_iat_max": "Fwd IAT Max",
    "fwd_iat_min": "Fwd IAT Min", "fwd_iat_mean": "Fwd IAT Mean", "fwd_iat_std": "Fwd IAT Std",
    "fwd_psh_flags": "Fwd PSH Flags", "fwd_urg_flags": "Fwd URG Flags", "fwd_header_len": "Fwd Header Length",
    "fwd_act_data_pkts": "Fwd Act Data Pkts", "fwd_seg_size_min": "Fwd Seg Size Min",
    "fwd_seg_size_avg": "Fwd Seg Size Avg", "bwd_pkts_s": "Bwd Packets/s",
    "tot_bwd_pkts": "Total Bwd packets", "totlen_bwd_pkts": "Total Length of Bwd Packet",
    "bwd_pkt_len_max": "Bwd Packet Length Max", "bwd_pkt_len_min": "Bwd Packet Length Min",
    "bwd_pkt_len_mean": "Bwd Packet Length Mean", "bwd_pkt_len_std": "Bwd Packet Length Std",
    "bwd_iat_tot": "Bwd IAT Total", "bwd_iat_max": "Bwd IAT Max", "bwd_iat_min": "Bwd IAT Min",
    "bwd_iat_mean": "Bwd IAT Mean", "bwd_iat_std": "Bwd IAT Std", "bwd_psh_flags": "Bwd PSH Flags",
    "bwd_urg_flags": "Bwd URG Flags", "bwd_header_len": "Bwd Header Length", "fin_flag_cnt": "FIN Flag Count",
    "syn_flag_cnt": "SYN Flag Count", "rst_flag_cnt": "RST Flag Count", "psh_flag_cnt": "PSH Flag Count",
    "ack_flag_cnt": "ACK Flag Count", "urg_flag_cnt": "URG Flag Count", "cwr_flag_count": "CWR Flag Count",
    "ece_flag_cnt": "ECE Flag Count", "down_up_ratio": "Down/Up Ratio", "pkt_size_avg": "Average Packet Size",
    "pkt_len_max": "Packet Length Max", "pkt_len_min": "Packet Length Min",
    "pkt_len_mean": "Packet Length Mean", "pkt_len_std": "Packet Length Std",
    "pkt_len_var": "Packet Length Variance", "init_fwd_win_byts": "FWD Init Win Bytes",
    "init_bwd_win_byts": "Bwd Init Win Bytes", "fwd_byts_b_avg": "Fwd Bytes/Bulk Avg",
    "fwd_pkts_b_avg": "Fwd Packets/Bulk Avg", "fwd_blk_rate_avg": "Fwd Bulk Rate Avg",
    "bwd_byts_b_avg": "Bwd Bytes/Bulk Avg", "bwd_pkts_b_avg": "Bwd Packets/Bulk Avg",
    "bwd_blk_rate_avg": "Bwd Bulk Rate Avg", "subflow_fwd_pkts": "Subflow Fwd Packets",
    "subflow_fwd_byts": "Subflow Fwd Bytes", "subflow_bwd_pkts": "Subflow Bwd Packets",
    "subflow_bwd_byts": "Subflow Bwd Bytes", "active_max": "Active Max", "active_min": "Active Min",
    "active_mean": "Active Mean", "active_std": "Active Std", "idle_max": "Idle Max",
    "idle_min": "Idle Min", "idle_mean": "Idle Mean", "idle_std": "Idle Std",
    "protocol": "Protocol", "src_port": "Src Port", "dst_port": "Dst Port"
}


## Loading the CSV dataset

In [27]:
sample_df = pd.DataFrame()
try:
    sample_df = pd.read_csv(SAMPLE_CSV_PATH, low_memory=False)
    print(f"Loaded {len(sample_df)} rows from TII-SSRC-23 sample.")
except FileNotFoundError:
    print(f"ERROR: Cannot find {SAMPLE_CSV_PATH}.")
    print("Please upload the sampled_200k.csv file.")
    raise
sample_df.head()

Loaded 200000 rows from TII-SSRC-23 sample.


,ACK Flag Count,Active Max,Active Mean,Active Min,Active Std,Average Packet Size,Bwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Header Length,Bwd IAT Max,...,Subflow Bwd Packets,Subflow Fwd Bytes,Subflow Fwd Packets,Total Bwd packets,Total Fwd Packet,Total Length of Bwd Packet,Total Length of Fwd Packet,Traffic Subtype,Traffic Type,URG Flag Count
0,0.0,0.0,0.0,0.0,0.0,1690.0,0.0,0.0,0.0,0.0,...,0.0,1801.0,1.0,0.0,4.0,0.0,5404.0,DoS URG,DoS,4.0
1,1.0,1234227.0,1234227.0,1234227.0,0.0,50.0,0.0,0.0,20.0,0.0,...,0.0,66.0,1.0,1.0,4.0,0.0,200.0,DoS PSH,DoS,0.0
2,1.0,0.0,0.0,0.0,0.0,500.0,0.0,0.0,20.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,500.0,DoS CWR,DoS,0.0
3,1.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,20.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,50.0,DoS ACK,DoS,0.0
4,0.0,0.0,0.0,0.0,0.0,58.5,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,1.0,1.0,39.0,39.0,Bruteforce DNS,Bruteforce,0.0


## Display Traffic Types and Subtypes

### Sampled TII-SSRC-23

In [28]:
print("====== LABEL ======")
print(sample_df['Label'].value_counts())
print("\n====== TRAFFIC TYPES ======")
print(sample_df['Traffic Type'].value_counts())
print("\n====== TRAFFIC SUBTYPES ======")
print(sample_df['Traffic Subtype'].value_counts())

====== LABEL ======
Label
Malicious    198699
Benign         1301
Name: count, dtype: int64

====== TRAFFIC TYPES ======
Traffic Type
DoS                      160226
Information Gathering     19548
Mirai                     11820
Bruteforce                 7105
Video                       870
Text                        209
Audio                       190
Background                   32
Name: count, dtype: int64

====== TRAFFIC SUBTYPES ======
Traffic Subtype
DoS RST                  19867
Information Gathering    19548
DoS ACK                  18564
DoS PSH                  18296
DoS URG                  18262
DoS CWR                  17920
DoS ECN                  17906
DoS SYN                  17758
DoS FIN                  16343
DoS UDP                   9753
DoS HTTP                  5518
Mirai DDoS DNS            4521
Bruteforce DNS            2872
Mirai DDoS SYN            2303
Mirai DDoS HTTP           1828
Mirai Scan Bruteforce     1809
Bruteforce Telnet         1361
Bruteforc

## Adding Additional Data

### Load, Label, and Preprocess Local CSVs

In [29]:
local_dfs = []
tii_columns = list(sample_df.columns)

for filename, (label, t_type, t_subtype) in LOCAL_CSV_LABELS.items():
    fpath = os.path.join(LOCAL_DATA_DIR, filename)
    if os.path.exists(fpath):
        try:
            df = pd.read_csv(fpath)
            # 1. Rename columns to match TII
            df.rename(columns=COL_MAPPING, inplace=True)

            # 2. Add Label metadata
            df['Label'] = label
            df['Traffic Type'] = t_type
            # Optional: maintain subtype if you want fine-grained control later
            if 'Traffic Subtype' in sample_df.columns:
                 df['Traffic Subtype'] = t_subtype

            # 3. Align columns exactly with TII data
            # This adds missing columns (filled with 0) and drops extra ones
            df_aligned = df.reindex(columns=tii_columns, fill_value=0)

            local_dfs.append(df_aligned)
            print(f"Loaded {filename}: {df_aligned.shape} -> mapped to {t_type}")
        except Exception as e:
             print(f"Error reading {filename}: {e}")
    else:
        print(f"Warning: Local file not found: {filename}")

if local_dfs:
    combined_df = pd.concat([sample_df] + local_dfs, ignore_index=True)
else:
    combined_df = sample_df
    print("No local data merged.")

print(f"Combined dataset shape: {combined_df.shape}")

Loaded Benign_Video.csv: (328, 82) -> mapped to Video
Loaded Benign_Audio.csv: (35, 82) -> mapped to Audio
Loaded Benign_Text.csv: (90, 82) -> mapped to Text
Loaded Benign_Web.csv: (14989, 82) -> mapped to Browsing
Loaded InfoGathering.csv: (18013, 82) -> mapped to Information Gathering
Loaded Bruteforce.csv: (209, 82) -> mapped to Bruteforce
Combined dataset shape: (233664, 82)


### Data Distribution after combination

In [30]:
print("====== LABEL ======")
print(combined_df['Label'].value_counts())
print("\n====== TRAFFIC TYPES ======")
print(combined_df['Traffic Type'].value_counts())

====== LABEL ======
Label
Malicious    216921
Benign        16743
Name: count, dtype: int64

====== TRAFFIC TYPES ======
Traffic Type
DoS                      160226
Information Gathering     37561
Browsing                  14989
Mirai                     11820
Bruteforce                 7314
Video                      1198
Text                        299
Audio                       225
Background                   32
Name: count, dtype: int64


## Data Cleaning

### Drop Identifiers

In [31]:
# 4.1. Drop sampling artifacts if they exist
cols_to_drop = ['_subtype', '_cluster', 'Flow ID', 'Timestamp', 'Src IP', 'Dst IP']
# Only drop if they are actually in the dataframe
actual_drop = [c for c in cols_to_drop if c in combined_df.columns]
combined_df.drop(columns=actual_drop, inplace=True)

### Infinity and NaNs

In [32]:
# Flow datasets often have infinite values in rate columns if duration is 0.
# We replace Inf with NaN, then drop or fill NaNs.
print("Checking for Infinity values...")
current_inf = np.isinf(combined_df.select_dtypes(include=np.number)).sum().sum()
print(f"Found {current_inf} Infinity values. Replacing with NaN.")
combined_df.replace([np.inf, -np.inf], np.nan, inplace=True)

Checking for Infinity values...
Found 0 Infinity values. Replacing with NaN.


### Drop NULLs

In [33]:
# Since we have lots of data, dropping rows with NaNs is usually safe
# unless a specific column is totally corrupted.
print(f"Rows before dropping NaNs: {len(combined_df)}")
combined_df.dropna(inplace=True)
print(f"Rows after dropping NaNs: {len(combined_df)}")

Rows before dropping NaNs: 233664
Rows after dropping NaNs: 233664


### Drop duplicates, if any

In [34]:
# Drop duplicates
before_dedup = len(combined_df)
combined_df.drop_duplicates(inplace=True)
print(f"Dropped {before_dedup - len(combined_df)} duplicates.")

Dropped 492 duplicates.


## Encoding Features
-  The 'Protocol' feature is categorical, despite being recognized as numerical.

In [35]:
print("Encoding Protocol...")
# Ensure it's int for consistency before string conversion
combined_df['Protocol'] = combined_df['Protocol'].astype(int).astype(str)

# Encode protocol
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
protocol_ohe = encoder.fit_transform(combined_df[['Protocol']])
feature_names = encoder.get_feature_names_out(['Protocol'])

# Safely concatenate
protocol_df = pd.DataFrame(protocol_ohe, columns=feature_names, index=combined_df.index)
combined_df = pd.concat([combined_df.drop(columns=['Protocol']), protocol_df], axis=1)

Encoding Protocol...


## Label Encoding

In [36]:
print("Encoding Labels...")
combined_df['Label'] = combined_df['Label'].map({'Benign': 0, 'Malicious': 1})

# Verify
print(combined_df['Label'].value_counts())

Encoding Labels...
Label
1    216430
0     16742
Name: count, dtype: int64


## Final Review

In [37]:
print("\nFinal Data Structure:")
print(combined_df.info())


Final Data Structure:
<class 'pandas.core.frame.DataFrame'>
Index: 233172 entries, 0 to 233663
Data columns (total 84 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ACK Flag Count              233172 non-null  float64
 1   Active Max                  233172 non-null  float64
 2   Active Mean                 233172 non-null  float64
 3   Active Min                  233172 non-null  float64
 4   Active Std                  233172 non-null  float64
 5   Average Packet Size         233172 non-null  float64
 6   Bwd Bulk Rate Avg           233172 non-null  float64
 7   Bwd Bytes/Bulk Avg          233172 non-null  float64
 8   Bwd Header Length           233172 non-null  float64
 9   Bwd IAT Max                 233172 non-null  float64
 10  Bwd IAT Mean                233172 non-null  float64
 11  Bwd IAT Min                 233172 non-null  float64
 12  Bwd IAT Std                 233172 non-null  float64
 

### Save the data

In [38]:
# Save as CSV (classic)
combined_df.to_csv(CLEAN_OUTPUT_CSV, index=False)
print(f"Saved CSV to {CLEAN_OUTPUT_CSV}")

# Save as Parquet (recommended for speed in next steps)
# Need to ensure column names are strings for parquet
combined_df.columns = combined_df.columns.astype(str)
combined_df.to_parquet(CLEAN_OUTPUT_PARQUET, index=False)
print(f"Saved Parquet to {CLEAN_OUTPUT_PARQUET}")

Saved CSV to /content/drive/MyDrive/Datasets/TII-SSRC-23/cleaned_data.csv
Saved Parquet to /content/drive/MyDrive/Datasets/TII-SSRC-23/cleaned_data.parquet
